In [1]:
import sys
sys.path.insert(0, '..\..')

In [2]:
from submissions.full_nn.dataLoaders import imageDataLoader
from submissions.full_nn.dataset import folliclesDataset

# Playground

In [7]:
from torch.utils.data import DataLoader

In [3]:
import glob
import pandas as pd
import imageio
from matplotlib import pyplot as plt
import torch

In [4]:
train_files = glob.glob("../../data/train/*.jpg")
test_files = glob.glob("../../data/test/*.jpg")
train_label = pd.read_csv("../../data/train/labels.csv")
test_label = pd.read_csv("../../data/test/labels.csv")

test_loader = imageDataLoader(test_files, test_label[["filename","xmin","xmax","ymin","ymax","label"]])
train_loader = imageDataLoader(train_files, train_label[["filename","xmin","xmax","ymin","ymax","label"]])
predict_loader = imageDataLoader(test_files, None)

In [5]:
train_loader = folliclesDataset(
    train_loader,
    local_path="../../data/train_dataset",
    window_size=(500,500),
    verbose=False,
    force_reload=False
)

In [6]:
len(train_loader)

4352

In [24]:
import numpy as np
from torch import nn
from torchvision.transforms import Resize, RandomHorizontalFlip, RandomVerticalFlip

In [99]:
def dataloader_collapse (x, random_flip=True):
    """dataloader_collate
    
    Function of collate for the DataLoader class use with the datasetFollicles class.
    This function is used to provide images from the datasetFollicles in a tensor for training task.

    Parameters
    ----------
    x: input data to collate, list containing tuple of a numpy array of size (h,w,3) and a dictionnary is expected
    random_flip: boolean, if True, random flip of the images are performed

    Output
    ------
    Tuple containing a tensor of size :
        (batch_size, *features) with features the size of the features data
        (batch_size, 5) with 5 the one-hot encoding of the label
    """

    Xs = []
    ys = []
    mean_ratio = np.array([data[1]["height"]/data[1]["width"] for data in x]).mean()

    if random_flip:
        random_hflip = RandomHorizontalFlip()
        random_vflip = RandomVerticalFlip()

    for data in x:
        # Getting data in tensor
        data_tensor = torch.tensor(data[0], dtype=torch.float32)
        data_tensor = torch.moveaxis(data_tensor, 2, 0)

        # Random transformations
        #if random_flip:
            #data_tensor = random_hflip(data_tensor)
            #data_tensor = random_vflip(data_tensor)

        # Getting label and box
        if data[1]["bbox_label"] is not None:
            bbox_label = {
                    "boxes":torch.tensor(data[1]["bbox"], dtype=torch.float32),
                    "labels":torch.tensor(data[1]["bbox_label"], dtype=torch.int64)
            }

            ys.append(bbox_label)
        else:
            y = None
        
        Xs.append(data_tensor)

    X = torch.stack(Xs)
    y = ys
    
    return X, y

In [100]:
train_dataloader = DataLoader(train_loader, batch_size=64, shuffle=True, collate_fn=dataloader_collapse)

In [101]:
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection import fasterrcnn_resnet50_fpn

In [102]:
model = fasterrcnn_resnet50_fpn(pretrained=True)
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, 5)

In [103]:
for x, y in train_dataloader:
    break

In [104]:
model(x,y)

{'loss_classifier': tensor(166.2936, grad_fn=<NllLossBackward0>),
 'loss_box_reg': tensor(0.4976, grad_fn=<DivBackward0>),
 'loss_objectness': tensor(167.4704, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>),
 'loss_rpn_box_reg': tensor(1.7708, grad_fn=<DivBackward0>)}

In [106]:
with torch.no_grad():
    model.eval()
    res = model(x)